# TAL aplicado al análisis del discurso de los medios de prensa 📰🤓🔥


El proyecto consiste en entrenar y evaluar varios modelos de clasificación supervisada capaz de clasificar una noticia.


- Hito Unidad 1 (29 de septiembre): Datasets de entrenamiento y test + primer modelo baseline



### índex

1. [Importación del dataset](a)
2. [Extracción de tópicos vía RE en URLs](b)
3. [Extracción de términos clave de columnas title+text](b)
4. [Clasificación del dataset de entrenamiento](c)

## Generando dataset de entrenamiento
El dataset se clasificará en función de las siguientes etiquetas:
- mundo = 0 
- economía = 1 
- política y conflictos = 2 
- ciencia y tecnología = 3
- catástrofes y accidentes = 4 
- cultura y arte = 5 
- deporte = 6 
- ecología y planeta = 7
- crimen, delitos y justicia = 8 
- salud = 9

In [3]:
import warnings
warnings.filterwarnings('ignore')

# Data manipulation
import re
import pandas as pd
pd.set_option("display.max_rows", None)
import numpy as np

# NLP
import spacy
from spacy.matcher import Matcher
nlp = spacy.load("es_core_news_sm")
print("versión de Spacy: ", spacy.__version__)

versión de Spacy:  3.1.3


### 1. Importación de dataset <a name="a"></a>

In [2]:
frames = [pd.read_csv("data/chile_2020-09.csv"), pd.read_csv("data/chile_2020-10.csv"),pd.read_csv("data/chile_2020-11.csv"), pd.read_csv("data/chile_2020-12.csv"), pd.read_csv("data/chile_2021-01.csv"), pd.read_csv("data/chile_2021-02.csv"), pd.read_csv("data/chile_2021-03.csv"), pd.read_csv("data/chile_2021-04.csv")]
dataset = pd.concat(frames)
dataset['content'] = dataset['title'] + " " + dataset['text'] 

print("Largo de dataset: ", len(dataset))
dataset= dataset[dataset['text'].notna()]
        
dataset.drop(columns=['country','year','date','id', 'id_journalist'], inplace=True)
dataset['topic'] = ""

Largo de dataset:  69702


### 2.  Extracción de tópicos vía RE en URL <a name="b"></a>

In [3]:
#identicando fuentes fuentes
set(dataset["media_outlet"])

{'ahoranoticiasmega',
 'biobiochile',
 'elciudadano',
 'elmostrador',
 'emol',
 'horas24',
 'latercera'}

In [4]:
print(dataset[dataset.media_outlet=="ahoranoticiasmega"]["url"].head(2))
print(dataset[dataset.media_outlet=="horas24"]["url"].head(2))
print(dataset[dataset.media_outlet=="elciudadano"]["url"].head(2))
print(dataset[dataset.media_outlet=="biobiochile"]["url"].head(2))
print(dataset[dataset.media_outlet=="elmostrador"]["url"].head(2))
print(dataset[dataset.media_outlet=="emol"]["url"].head(2))
print(dataset[dataset.media_outlet=="latercera"]["url"].head(2))

268    https://www.meganoticias.cl/nacional/312353-si...
269    https://www.meganoticias.cl/nacional/312350-cy...
Name: url, dtype: object
0    https://www.24horas.cl/coronavirus/tia-pikachu...
3    https://www.24horas.cl/nacional/rm-alcanza-sus...
Name: url, dtype: object
205    https://www.elciudadano.com/chile/el-avance-de...
206    https://www.elciudadano.com/latinoamerica/colo...
Name: url, dtype: object
29    https://www.biobiochile.cl/noticias/nacional/c...
30    https://www.biobiochile.cl/noticias/nacional/r...
Name: url, dtype: object
228    https://www.elmostrador.cl/noticias/pais/2020/...
230    https://www.elmostrador.cl/dia/2020/09/01/dipu...
Name: url, dtype: object
496    https://www.emol.com/noticias/Economia/2020/09...
497    https://www.emol.com/noticias/Economia/2020/09...
Name: url, dtype: object
209    https://www.latercera.com/politica/noticia/el-...
210    https://www.latercera.com/politica/noticia/cam...
Name: url, dtype: object


Las urls de _Ahoranoticiasmega, Horas24 y El Ciudadano_ son similares.
La url de _La Tercera_ posee dos tópicos, sin embargo, el segundo tópico es irrelevante.

In [5]:
#Generamos df1 concatenando ahoranoticiasmega, horas24 y elciudadano
medios = [dataset.loc[dataset['media_outlet']=="ahoranoticiasmega"],
          dataset.loc[dataset['media_outlet']=="horas24"],
          dataset.loc[dataset['media_outlet']=="elciudadano"],
          dataset.loc[dataset['media_outlet']=="latercera"]]
df1 = pd.concat(medios)
for index, row in df1.iterrows():
    url=row['url']
    obj = re.findall('(\w+)://([\w\-\.]+)/([\w\-]+).([\w\-]+)', url) 
    topic=obj[0][2]
    row['topic']=topic.lower()

Las urls de _Biobiochile, El Mostrador, Emol_ poseen dos columnas de tópicos. La primera columna entrega información irrelevante. 

In [6]:
medios2 = [dataset.loc[dataset['media_outlet']=="biobiochile"],
           dataset.loc[dataset['media_outlet']=="elmostrador"],
           dataset.loc[dataset['media_outlet']=="emol"]] 
df2 = pd.concat(medios2)
for index, row in df2.iterrows():
    url=row['url']
    obj = re.findall('(\w+)://([\w\-\.]+)/([\w\-]+).([\w\-]+)', url)
    topic = obj[0][3]
    row['topic']=topic.lower()

In [7]:
df = pd.concat([df1,df2])
np.sort(df['topic'].unique())

array(['2020', '2021', 'actualidad', 'animal', 'artes', 'autos',
       'calidad-de-vida', 'cartas-ciudadanas', 'chile',
       'ciencia-tecnologia', 'ciudadanos-al-poder', 'columnas',
       'conversacioneslt', 'coronavirus', 'culto', 'data', 'deportes',
       'derechos-humanos', 'destacados-cultura', 'eclipse-2020',
       'economia', 'editorial', 'educacion', 'el-deportivo',
       'entrevistas', 'especiales', 'espectaculos', 'genero',
       'internacional', 'justicia', 'la-tercera-domingo', 'la-tercera-pm',
       'la-tercera-tv', 'latinoamerica', 'mapuche', 'masdeco',
       'medio-ambiente', 'mexico', 'mineria', 'mouse', 'multimedia',
       'mundo', 'nacional', 'negocios', 'noticiasbbc', 'opinion', 'pais',
       'paula', 'peru', 'plebiscito-chile-elige', 'podcast', 'politica',
       'portada', 'portada-tv', 'primarias-2020', 'proceso-constituyente',
       'programas', 'pueblos', 'pulso', 'pulso-trader', 'que-pasa',
       'reconstitucion', 'redes-sociales', 'regiones',
    

In [8]:
df['label'] = ""
# mundo
df.loc[df['topic'] == 'mundo',                  'label'] = 0
df.loc[df['topic'] == 'internacional',          'label'] = 0
# economia
df.loc[df['topic'] == 'economia',               'label'] = 1
df.loc[df['topic'] == 'pulso-trader',           'label'] = 1
# política y conflictos
df.loc[df['topic'] == 'politica',               'label'] = 2
df.loc[df['topic'] == 'data',                   'label'] = 2
df.loc[df['topic'] == 'plebiscito-chile-elige', 'label'] = 2
df.loc[df['topic'] == 'primarias-2020',         'label'] = 2
df.loc[df['topic'] == 'proceso-constituyente',  'label'] = 2
df.loc[df['topic'] == 'pulso',                  'label'] = 2
df.loc[df['topic'] == 'reconstitucion',         'label'] = 2
# ciencia y tecnologia
df.loc[df['topic'] == 'tecnologia',             'label'] = 3
df.loc[df['topic'] == 'tecnologia-2',           'label'] = 3
df.loc[df['topic'] == 'ciencia-tecnologia',     'label'] = 3
df.loc[df['topic'] == 'redes-sociales',         'label'] = 3
# accidentes 
# arte y cultura
df.loc[df['topic'] == 'artes',                  'label'] = 5
df.loc[df['topic'] == 'destacados-cultura',     'label'] = 5
df.loc[df['topic'] == 'educacion',              'label'] = 5
df.loc[df['topic'] == 'espectaculos',           'label'] = 5
df.loc[df['topic'] == 'turismo',                'label'] = 5
# deportes
df.loc[df['topic'] == 'deportes',               'label'] = 6
df.loc[df['topic'] == 'el-deportivo',           'label'] = 6
# ecologia y planeta
df.loc[df['topic'] == 'medio-ambiente',         'label'] = 7
df.loc[df['topic'] == 'animal',                 'label'] = 7
df.loc[df['topic'] == 'eclipse-2020',           'label'] = 7
# crimen, delitos y justicia
df.loc[df['topic'] == 'justicia',               'label'] = 8
df.loc[df['topic'] == 'reportaje-investigacion','label'] = 8
df.loc[df['topic'] == 'trabajo',                'label'] = 8
# salud
df.loc[df['topic'] == 'salud',                  'label'] = 9
df.loc[df['topic'] == 'calidad-de-vida',        'label'] = 9

display(df['label'].value_counts())
# nos quedamos sólo con las noticias etiquetadas. para clasificar df1 utilizaremos otro metodo
df1 = df.loc[df['label']==''] 
df = df[df['label']!='']

     43190
0     8564
6     7994
2     3329
1     2670
8      537
9      362
3      292
5      239
7      162
Name: label, dtype: int64

Tenemos muchas noticias no clasificadas, por lo que es necesario utilizar otro mecanísmo para clasificar las noticias restantes.

### 3.  Extracción de términos clave de columna title+text <a name="b"></a>

In [9]:
matcher = Matcher(nlp.vocab)

# Pattern 1: NOUN de NOUN
pattern_1 = [{"POS": "NOUN"},{"LOWER": "de"}, {"POS": "NOUN"}]
matcher.add("NOUN-de-NOUN", [pattern_1])

# Pattern 2: NOUN ADJ
pattern_2 = [{"POS": "NOUN"}, {"POS": "ADJ"}]
matcher.add("NOUN-ADJ", [pattern_2])

    Bloque de alto coste computacional ⬇️

In [10]:
df1['terms'] = ""
for index, row in df1.iterrows():
    doc = nlp(row['content'].lower())

    matches = matcher(doc)
    categories = ""
    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]  # Get string representation
        span = doc[start:end]  # The matched span
        categories = categories + span.text + "; "
    
    row['terms'] = categories
    
display(df1.head(3))

,media_outlet,url,title,text,content,topic,label,terms
268,ahoranoticiasmega,https://www.meganoticias.cl/nacional/312353-si...,Temblor de menor intensidad afecta nuevamente ...,"La noche de este martes 01 de septiembre, a la...",Temblor de menor intensidad afecta nuevamente ...,nacional,,sismo de magnitud; zona norte; centro sismológ...
269,ahoranoticiasmega,https://www.meganoticias.cl/nacional/312350-cy...,CyberDay 2020: Consolas y videojuegos son los ...,"En la presente edición del CyberDay, el rubro ...",CyberDay 2020: Consolas y videojuegos son los ...,nacional,,costo de producción; consumidores chilenos; ru...
270,ahoranoticiasmega,https://www.meganoticias.cl/nacional/312347-on...,Sismos en el norte: Expertos aclaran que exist...,La Oficina Nacional de Emergencia del Minister...,Sismos en el norte: Expertos aclaran que exist...,nacional,,energía acumulada; centro sismológico; región ...


In [11]:
from collections import Counter
df1["most_common"] = ""
df1["second_common"] = ""

for index, row in df1.iterrows():
    split_it = row['terms'].split("; ")
    counter = Counter(split_it)
    row['most_common'] = counter.most_common()[0][0]
    try:
        row['second_common'] = counter.most_common()[1][0]
    except IndexError:
        row['second_common'] = counter.most_common()[0][0]

display(df1.head(3))

,media_outlet,url,title,text,content,topic,label,terms,most_common,second_common
268,ahoranoticiasmega,https://www.meganoticias.cl/nacional/312353-si...,Temblor de menor intensidad afecta nuevamente ...,"La noche de este martes 01 de septiembre, a la...",Temblor de menor intensidad afecta nuevamente ...,nacional,,sismo de magnitud; zona norte; centro sismológ...,región de atacama,hora local
269,ahoranoticiasmega,https://www.meganoticias.cl/nacional/312350-cy...,CyberDay 2020: Consolas y videojuegos son los ...,"En la presente edición del CyberDay, el rubro ...",CyberDay 2020: Consolas y videojuegos son los ...,nacional,,costo de producción; consumidores chilenos; ru...,costo de producción,consumidores chilenos
270,ahoranoticiasmega,https://www.meganoticias.cl/nacional/312347-on...,Sismos en el norte: Expertos aclaran que exist...,La Oficina Nacional de Emergencia del Minister...,Sismos en el norte: Expertos aclaran que exist...,nacional,,energía acumulada; centro sismológico; región ...,energía acumulada,centro sismológico


**Para no ejecutar siempre la celda de alto costo computacional, almacenamos la data procesada en un nuevo csv con los términos claves y luego solo importamos el csv**

In [12]:
df1.to_csv("keysTerms.csv", index=False) 

In [13]:
df2=pd.read_csv("keysTerms.csv")
df2.replace(np.nan,"", inplace=True)

Etiquetamos los términos más comunes que pertenezcan a una de las categorías

In [14]:
#4 catastrofes y accidentes(0)
df2.loc[df2['most_common'] == 'incendio forestal',                  'label'] = 4
df2.loc[df2['most_common'] == 'accidente de tránsito',              'label'] = 4
df2.loc[df2['most_common'] == 'incendios forestales',               'label'] = 4
df2.loc[df2['most_common'] == 'centro sismológico',                 'label'] = 4
df2.loc[df2['most_common'] == 'movimiento telúrico',                'label'] = 4
df2.loc[df2['most_common'] == 'víctimas fatales'                    'label'] = 4
df2.loc[df2['second_common'] == 'tormentas eléctricas'              'label'] = 4
df2.loc[df2['second_common'] == 'descarga eléctrica'                'label'] = 4
df2.loc[df2['second_common'] == 'centro sismológico'                'label'] = 4
df2.loc[df2['second_common'] == 'accidente de tránsito'             'label'] = 4
df2.loc[df2['second_common'] == 'movimiento telúrico'               'label'] = 4
df2.loc[df2['second_common'] == 'incendio forestal'                 'label'] = 4
df2.loc[df2['second_common'] == 'sismo de magnitud'                 'label'] = 4
df2.loc[df2['second_common'] == 'víctima fatal'                     'label'] = 4
df2.loc[df2['second_common'] == 'accidentes de tránsito'            'label'] = 4
#5 cultura y arte(179)
df2.loc[df2['most_common'] == 'consulta indígena',                  'label'] = 5
df2.loc[df2['second_common'] == 'pueblos originarios',              'label'] = 5
#7 ecología y planeta(168)
df2.loc[df2['most_common'] == 'cambio climático',                   'label'] = 7
df2.loc[df2['most_common'] == 'eclipse solar',                      'label'] = 7
df2.loc[df2['most_common'] == 'agua potable',                       'label'] = 7
df2.loc[df2['most_common'] == 'eclipse total',                      'label'] = 7
df2.loc[df2['most_common'] == 'aguas andinas',                      'label'] = 7
df2.loc[df2['most_common'] == 'áreas verdes',                       'label'] = 7
df2.loc[df2['most_common'] == 'evaluación ambiental',               'label'] = 7
df2.loc[df2['most_common'] == 'sistema solar',                      'label'] = 7
df2.loc[df2['most_common'] == 'territorio antártico',               'label'] = 7
df2.loc[df2['second_common'] == 'cambio climático',                 'label'] = 7
df2.loc[df2['second_common'] == 'eclipse solar',                    'label'] = 7
df2.loc[df2['second_common'] == 'agua potable',                     'label'] = 7
#8 crimen, delitos y justicia
df2.loc[df2['most_common'] == 'prisión preventiva',                 'label'] = 8
df2.loc[df2['most_common'] == 'acusación constitucional',           'label'] = 8 #no estoy seguro
df2.loc[df2['most_common'] == 'corte de apelaciones',               'label'] = 8
df2.loc[df2['most_common'] == 'años de cárcel',                     'label'] = 8
df2.loc[df2['most_common'] == 'ataque incendiario',                 'label'] = 8
df2.loc[df2['most_common'] == 'abuso sexual',                       'label'] = 8
df2.loc[df2['most_common'] == 'brigada de homicidios',              'label'] = 8
df2.loc[df2['most_common'] == 'hechos de violencia',                'label'] = 8
df2.loc[df2['most_common'] == 'armas de fuego',                     'label'] = 8
df2.loc[df2['most_common'] == 'crimen organizado',                  'label'] = 8
df2.loc[df2['most_common'] == 'arma de fuego',                      'label'] = 8
df2.loc[df2['most_common'] == 'homicidio frustrado',                'label'] = 8
df2.loc[df2['most_common'] == 'acoso sexual',                       'label'] = 8
df2.loc[df2['most_common'] == 'juicio oral',                        'label'] = 8
df2.loc[df2['most_common'] == 'femicidio frustrado',                'label'] = 8

#9 salud
df2.loc[df2['most_common'] == 'casos activos',                      'label'] = 9
df2.loc[df2['most_common'] == 'seremi de salud',                    'label'] = 9
df2.loc[df2['most_common'] == 'salud pública',                      'label'] = 9
df2.loc[df2['most_common'] == 'colegio médico',                     'label'] = 9
df2.loc[df2['most_common'] == 'medidas sanitarias',                 'label'] = 9
df2.loc[df2['most_common'] == 'proceso de vacunación',              'label'] = 9
df2.loc[df2['most_common'] == 'ensayos clínicos',                   'label'] = 9
df2.loc[df2['most_common'] == 'autoridad sanitaria',                'label'] = 9
df2.loc[df2['most_common'] == 'casos confirmados',                  'label'] = 9
df2.loc[df2['most_common'] == 'salud mental',                       'label'] = 9
df2.loc[df2['most_common'] == 'contactos estrechos',                'label'] = 9
df2.loc[df2['most_common'] == 'informe epidemiológico',             'label'] = 9
df2.loc[df2['most_common'] == 'estado de salud',                    'label'] = 9
df2.loc[df2['most_common'] == 'vacunación masiva',                  'label'] = 9
df2.loc[df2['second_common'] == 'salud pública',                    'label'] = 9
df2.loc[df2['second_common'] == 'colegio médico',                   'label'] = 9
df2.loc[df2['second_common'] == '',                                 'label'] = 9
df2.loc[df2['second_common'] == '',                                 'label'] = 9
#1 economia(1772)
df2.loc[df2['most_common'] == 'retiro de fondos',                   'label'] = 1
df2.loc[df2['most_common'] == 'fondos de pensiones',                'label'] = 1
df2.loc[df2['most_common'] == 'millones de euros',                  'label'] = 1
df2.loc[df2['most_common'] == 'fondos previsionales',               'label'] = 1
df2.loc[df2['most_common'] == 'salario mínimo',                     'label'] = 1
df2.loc[df2['most_common'] == 'pago de impuestos',                  'label'] = 1
df2.loc[df2['most_common'] == 'crisis económica',                   'label'] = 1
df2.loc[df2['most_common'] == 'economía chilena',                   'label'] = 1
df2.loc[df2['most_common'] == 'metal rojo',                         'label'] = 1
df2.loc[df2['most_common'] == 'mercado laboral',                    'label'] = 1
df2.loc[df2['second_common'] == 'impuestos internos',               'label'] = 1



In [15]:
df = pd.concat([df,df2])
#df.drop(columns=['most_common','second_common','terms'], inplace=True)
df['label'].value_counts()

     39198
0     8564
6     7994
2     3329
1     2932
8     2144
9     1928
7      414
5      306
3      292
4      238
Name: label, dtype: int64

### 4. Identificar subtópicos relacionados a los tópicos faltantes.

Para obtener un dataset balanceado, falta identificar noticias de:
* catástrofes y accidentes (4)
* ecología y planeta (7)
* cultura y arte (5)
* economía (1)

El título de una noticia representa información valiosa para conocer de que trata la noticia, por lo tanto, asociamos una palabra específica dentro del título con un tópico específico. Por ejemplo la palabra temblor, se asocia directamente con el tópico catástrofes y accidentes (4)

In [16]:
df2[df2['title'].str.contains("(?i)temblor")]['title']

0        Temblor de menor intensidad afecta nuevamente ...
8        "Con mucho susto": Alcalde de Huasco por segui...
10       Fuerte temblor de 6,3 grados afecta a la regió...
12                     Nuevo temblor se registra en Huasco
14             Temblor se registra en la región de Atacama
32       SHOA descarta riesgo de tsunami tras nuevo tem...
33       Imágenes revelan el daño que provocaron los te...
36       Onemi llama a la tranquilidad luego de los tem...
37       Nuevo temblor se registra en la zona norte del...
38       Revisa los videos de los temblores que afectar...
39       Temblor magnitud 7.0 afectó a la zona norte de...
73        Temblor afecta nuevamente a la Región de Atacama
74       De nube tóxica a fenómeno por temblor en el no...
112                    Nuevo temblor se registra en Huasco
117                  Temblor afecta a la región de Atacama
125      Nuevo temblor se registra en la región de Atacama
210      Temblor en la región de Coquimbo: Revisa regis.

In [17]:
df2 = df.loc[df['label']=='']
df = df[df['label']!='']

df2.loc[df2['title'].str.contains("(?i)temblor|erupción|incendio|catástrofe|accidente"), 'label'] = 4 
df2.loc[df2['title'].str.contains("(?i)tecnol|científic|ciencia"), 'label'] = 3 
df2.loc[df2['title'].str.contains("(?i)ecología|planeta|contamina|atmósfera|eclipse|tierra|reciclaje|vegetación|paisaje|animal|verdes"), 'label'] = 7
df2.loc[df2['title'].str.contains("(?i)cultura|pintura|museo|música|patrimonio|literatura|arqueólogo|arqueología|termopolio|geólogo"), 'label'] = 5 #arte no por que toma otras palabras como valuarte, etc

df2.dropna(subset = ["label"], inplace=True)
df2['label'].value_counts()

     37259
4     1106
7      403
3      231
5      199
Name: label, dtype: int64

In [18]:
df = pd.concat([df,df2])
df['label'].value_counts()

     37259
0     8564
6     7994
2     3329
1     2932
8     2144
9     1928
4     1344
7      817
3      523
5      505
Name: label, dtype: int64

In [19]:
df.replace("", np.nan, inplace=True)
df.dropna(subset = ["label"], inplace=True)

df = df.astype({"label": str})
df.replace("0.0","mundo", inplace=True)
df.replace("1.0","economía", inplace=True)
df.replace("2.0","política y conflictos", inplace=True)
df.replace("3.0","ciencia y tecnología", inplace=True)
df.replace("4.0","catástrofes y accidentes", inplace=True)
df.replace("5.0","cultura y arte", inplace=True)
df.replace("6.0","deporte", inplace=True)
df.replace("7.0","ecología y planeta", inplace=True)
df.replace("8.0","crimen, delitos y justicia", inplace=True)
df.replace("9.0","salud", inplace=True)

In [20]:
df.to_csv("dataEtiquetada.csv", index=False) 
df['label'].value_counts()

mundo                         8564
deporte                       7994
política y conflictos         3329
economía                      2932
crimen, delitos y justicia    2144
salud                         1928
catástrofes y accidentes      1344
ecología y planeta             817
ciencia y tecnología           523
cultura y arte                 505
Name: label, dtype: int64

In [ ]:
# Duplicación de noticias

In [4]:
df = pd.read_csv("dataEtiquetada.csv")
df['label'].value_counts()

mundo                         8564
deporte                       7994
política y conflictos         3329
economía                      2932
crimen, delitos y justicia    2144
salud                         1928
catástrofes y accidentes      1344
ecología y planeta             817
ciencia y tecnología           523
cultura y arte                 505
Name: label, dtype: int64

In [5]:
df7 = df.loc[df['label']=='ecología y planeta']
df5 = df.loc[df['label']=='cultura y arte']
df4 = df.loc[df['label']=='catástrofes y accidentes']
df3 = df.loc[df['label']=='ciencia y tecnología']
df = pd.concat([df,df7,df3,df5,df7,df3,df5,df7,df5,df4,df3])
def sampling_k_elements(group, k=2060):
    if len(group) < k:
        return group
    return group.sample(k)

df = df.groupby('label').apply(sampling_k_elements).reset_index(drop=True)
df['label'].value_counts()

catástrofes y accidentes      2060
ciencia y tecnología          2060
crimen, delitos y justicia    2060
deporte                       2060
ecología y planeta            2060
economía                      2060
mundo                         2060
política y conflictos         2060
cultura y arte                2020
salud                         1928
Name: label, dtype: int64

In [6]:
df.to_csv("dataEtiquetada.csv", index=False)